 # Cuaderno 21: Planificación de líneas

*Este cuaderno contiene material tomado del Trabajo de Integración Curricular de Darlyn Ludeña, en la carrera de Ingeniería Matemática en la Escuela Politécnica Nacional (Quito, 2022).*

El problema de *planificación de líneas* consiste en seleccionar, de entre un *pool* de líneas posibles, aquellas líneas necesarias para cubrir la demanda en cada arista de la red de transporte a un costo mínimo. A cada línea seleccionada se le debe asignar una frecuencia de operación, que indica el número de viajes programados para esa línea dentro del horizonte de tiempo.

En el presente ejemplo vamos a suponer que se tiene dado lo siguiente:
* una red de transporte $D=(V,A)$ (grafo dirigido);
* un conjunto $L$ de posibles líneas (*line pool*);
* un conjunto $F$ de valores posibles para las frecuencias (enteros positivos);
* un vector de costos de operación $c \in \mathbb{R}^{L \times F}$, donde $c_{lf}$ es el costo de operar la línea $l \in L$ con una  frecuencia $f \in F$; y,
* un vector de frecuencias mínimas $f_{ij}^{\min} \in \mathbb{R}^A$ y uno de frecuencias máximas $f_{ij}^{\max} \in \mathbb{R}^A$ asociados a cada arco $(i,j) \in A$.

Para la formulación del modelo emplearemos variables de decisión binarias $x_{lf}$, con $l \in L$ y $f \in F$, que indican si la línea $l$ es operada con la frecuencia $f$.

Con esta variables, el modelo puede formularse de la siguiente manera:

\begin{align*}
\min &\sum_{l \in L}  \sum_{f \in F} c_{lf} x_{lf}\\
&\mbox{s.r.}\\\\
&f_{ij}^{\min}\leq\sum_{l \in L : (i,j) \in l}\sum_{f \in F} f x_{lf} \leq f_{ij}^{\max}, \quad \forall (i,j) \in A, \\
&\sum_{f \in F} x_{lf} \leq 1, \quad \forall l \in L, \\
& x_{lf} \in \{0, 1\}, \quad \forall l \in L, f \in F.
\end{align*}

La función objetivo a minimizar consiste en la suma sobre todas las posibles líneas $l \in L$ y sobre todas las frecuencias $f \in F$ del producto de la variable de decisión $x_{lf}$ por el coeficiente de costo $c_{lf}$ correspondiente a la operación de $l$ en la frecuencia $f$. De esta manera, se busca minimizar el costo total de la operación de todas las líneas.

La primera familia de restricciones requiere que, para cada arco $(i,j) \in A$, la suma de las frecuencias de las líneas seleccionadas que contienen a $(i,j)$ esté dentro del rango factible $[f_{ij}^{\min}; f_{ij}^{\max}]$. Notar que al requerir que esta suma sea mayor o igual a $f_{ij}^{\min}$ se busca satisfacer la demanda de transporte estimada sobre $(i,j)$. Por otra parte, se establece que esta suma sea menor o igual a $f_{ij}^{\max}$ para respetar las restricciones técnicas de capacidad de circulación de las vías.

La segunda familia de restricciones especifica que para cada línea se puede seleccionar a lo más una frecuencia de operación. Si para una línea no se seleccionan frecuencias de operación, significa que la línea no será operada en la red. 

Implementaremos este modelo empleando el API Python de Gurobi, sobre la instancia que se indica a continuación:

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# Arcos, frecuencias mínima y máxima
A, fmin,fmax = gp.multidict({
  (1, 2): (2,5),
  (2, 3): (2,6),
  (3, 4): (2,10),
  (4, 3): (2,10),
  (4, 8):  (1,15),
  (5, 1):  (1,15),
  (6, 5):  (2,14),
  (4, 5):  (2,15),
  (5, 4):  (2,15),
  (7, 8):  (2,15),
  (2, 4):  (2,15),  
  (6, 4): (2,15), 
  (8, 6):  (2,15),
  (3, 7):  (1,20), 
  (4, 7): (2,15),
  (1, 4):  (2,18), 
  (8, 5):  (1,20)})

# Line pool, con los arcos para cada línea:
L = {1 : [(1, 2), (2, 3), (3, 7), (7, 8)],
         2 : [(1, 4), (4, 8)],
         3 : [(8, 6), (6, 5), (5, 1)], 
         4 : [(4, 3), (3, 7), (7, 8)],
         5 : [(2, 4), (4, 8), (8, 5), (5, 1)],
         6 : [(8, 6), (6, 4), (4, 5), (5, 1)],
         7 : [(1, 2), (2, 3), (3, 4)],
         8 : [(6, 4), (4, 3), (3, 7), (7, 8)],
         9 : [(1, 4), (4, 7)],
        10 : [(5, 4), (4, 7), (7, 8)]}

# Verificamos que los arcos de todas las líneas sean arcos de la red
# caso contrario, se levanta una excepción
for l in L:
    for e in L[l]:
        assert(e in A)

# Frecuencias de operación posibles
F = range(1,4)

# Costos de operación para cada línea y frecuencia
cost = {  (1,1) : 15,  (1,2) : 30,  (1,3) : 45,
          (2,1) : 10,  (2,2) : 25,  (2,3) : 35,
          (3,1) :  5,  (3,2) : 12,  (3,3) : 19,
          (4,1) : 12,  (4,2) : 28,  (4,3) : 36,
          (5,1) : 21,  (5,2) : 45,  (5,3) : 75,
          (6,1) : 14,  (6,2) : 30,  (6,3) : 50,
          (7,1) : 25,  (7,2) : 48,  (7,3) : 85,
          (8,1) : 30,  (8,2) : 62,  (8,3) : 100,
          (9,1) : 16,  (9,2) : 35,  (9,3) : 55,
         (10,1) : 22, (10,2) : 46, (10,3) : 66}

Ahora construimos a partir de los datos anteriores una lista `V` con los nodos de la red de transporte y un diccionario `La` que guarde para cada arco $(i,j) \in A$ la lista `Le[i,j]` de las líneas que lo contienen.

In [ ]:
# lista V con los nodos del grafo
V = gp.tuplelist(set([i for (i,j) in A] + [j for (i,j) in A]))

# diccionario con líneas que contienen a cada arco
La = {(i,j) : [l for l in L.keys() if (i,j) in L[l]] for (i,j) in A}

print('V = {}'.format(V))
print('La = {}'.format(La))

Podemos graficar la red empleando las bibliotecas `networkx` y `matplotlib`:

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib as mp
G = nx.MultiDiGraph()
G.add_nodes_from(list(V))
# Iteramos sobre todas las líneas:
for l in L:
    # y sobre todos los arcos de cada línea
    for (i,j) in L[l]:
        # agregamos cada arco al multigrafo, registrando su línea como propiedad
        G.add_edge(i, j, linea=l)
# Diccionario con las posiciones de los nodos        
pos = {1:(-5,6),2:(-2,10),3:(4,10),4:(1,6),
       5:(-2,0),6:(1,2),7:(7,6),8:(4,0)}
plt.figure(figsize=(8,5))
nx.draw_networkx(G, pos, node_size=500,
                       alpha=1, edgecolors = 'black',
                       node_color='white', edgelist=[],
                       font_size=10)
ax = plt.gca()
colores = ['w', 'b', 'r', 'g', 'lime', 'm', 'c', 'saddlebrown', 'indigo', 'orange', 'gray']
# LMT : usar la línea como índice de color
lin = nx.get_edge_attributes(G,'linea')
for e in G.edges:
    # recuperamos la línea del arco, para usarla como color
    ax.annotate("",
                xy=pos[e[1]], xycoords='data',
                xytext=pos[e[0]], textcoords='data',
                arrowprops=dict(arrowstyle="->", color=colores[lin[e]],
                                shrinkA=12, shrinkB=12,
                                patchA=None, patchB=None,
                                connectionstyle="arc3,rad=rrr".replace('rrr',str(0.2*e[2])
                                ),
                                ),
                )
plt.axis('off')   
plt.show()

Se tienen 10 líneas, con los siguientes recorridos:

* La línea 1 de color azul que recorre las paradas 1,2,3,7,8;
* La línea 2 de color rojo que recorre las paradas 1,4,8;
* La línea 3 de color verde que recorre las paradas 8,6,5,1;
* La línea 4 de color verde claro que recorre las paradas 4,3,7,8;
* La línea 5 de color fucsia que recorre las paradas 2,4,8,5,1;
* La línea 6 de color celeste que recorre las paradas 8,6,4,5,1;
* La línea 7 de color café que recorre las paradas 1,2,3,4;
* La línea 8 de color morado que recore las paradas 6,4,3,7,8;
* La línea 9 de color amarillo que recorre las paradas 1,4,7;y,
* La línea 10 de color plomo que recorre las paradas 5,4,7,8.

Con los datos anteriores contruiremos el modelo de programación lineal entera. Empezamos por definir el objeto modelo y las variables de decisión binarias:

In [ ]:
m = gp.Model('planificacion-lineas')

x = m.addVars(L, F, vtype = GRB.BINARY, name="x")

Construimos ahora la función objetivo:

In [ ]:
# minimizar los costos de operación de líneas
m.setObjective(x.prod(cost, '*','*'), GRB.MINIMIZE)

Implementaremos ahora las restricciones del modelo:

1. Para cada arco $(i,j) \in A$, la suma de las frecuencias de las líneas que pasan por $(i,j)$ debe estar entre $f_{ij}^{\min}$ y $f_{ij}^{\max}$:

In [ ]:
# Frecuencia mínima por arco
m.addConstrs(((gp.quicksum([f*x[l,f] for l in La[i,j]for f in F ]) 
                         >= fmin[i,j]) for (i,j) in A), "freq_min")

# Frecuencia máxima por arco
m.addConstrs(((gp.quicksum([f*x[l,f] for l in La[i,j]for f in F ]) 
                         <= fmax[i,j]) for (i,j) in A), "freq_max")

2. Para cada una de las líneas, debe seleccionarse a lo más una frecuencia.

In [ ]:
m.addConstrs((x.sum(l,'*')<=1 for l in L), "freq_por_linea")

Exportamos el modelo a un archivo en formato `lp`:

In [ ]:
m.write("modelo_planif_lineas.lp")

Finalmente, resolvemos el modelo:

In [ ]:
# Calcular la solución óptima
m.optimize()

Mostramos a continuación la solución encontrada:

In [ ]:
for l in L:
    for f in F:
        if x[l, f].x >= 0.99:
            print("La línea {} se opera con frecuencia {}, costo {}.".format(l, f, cost[l,f]))

print('El costo total de operación de las líneas seleccionadas es: {}'.format(m.objVal))           

También es de interés mostrar el acumulado de las frecuencias de las líneas seleccionadas sobre cada arco de la red: 

In [ ]:
# Diccionario de frecuencias asignadas a cada línea en la solución
Fsol={l : f for l in L for f in F if x[l, f].x >= 0.99}

# Diccionario con las frecuencias acumuladas sobre cada arco
Farco = {(i,j) : sum(Fsol[l] for l in La[i,j] if l in Fsol.keys()) for i,j in A}

for (i,j) in A:
    print('({},{}): min= {}; sol= {}; max= {}'.format(i,j,fmin[i,j], Farco[i,j], fmax[i,j]))
               

Por último, pueden graficarse las líneas seleccionadas empleando las bibliotecas `networkx` y `matplotlib`:

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib as mp
G = nx.MultiDiGraph() 
G.add_nodes_from(list(V))
# Iteramos sobre todas las líneas:
for l in Fsol.keys():
    # y sobre todos los arcos de cada línea
    for (i,j) in L[l]:
        # agregamos cada arco al multigrafo, registrando su linea como propiedad
        G.add_edge(i, j, linea=l)

pos = {1:(-5,6),2:(-2,10),3:(4,10),4:(1,6),
       5:(-2,0),6:(1,2),7:(7,6),8:(4,0)}
plt.figure(figsize=(8,5))
nx.draw_networkx(G, pos, node_size=500,
                       alpha=1, edgecolors = 'black',
                       node_color='white', edgelist=[],
                       font_size=10)
ax = plt.gca()
colores = ['w', 'b', 'r', 'g', 'lime', 'm', 'c', 'saddlebrown', 'indigo', 'orange', 'gray']
lin = nx.get_edge_attributes(G,'linea')
for e in G.edges:
    # recuperamos la linea del arco, para usarla como color
    ax.annotate("",
                xy=pos[e[1]], xycoords='data',
                xytext=pos[e[0]], textcoords='data',
                arrowprops=dict(arrowstyle="->", color=colores[lin[e]],
                                shrinkA=14, shrinkB=14,
                                patchA=None, patchB=None,
                                connectionstyle="arc3,rad=rrr".replace('rrr',str(0.2*e[2])
                                ),
                                ),
                )
plt.axis('off')   
plt.show()